<h3>Objective</h3>
<p>I have now tried the grid_search and tried the k fold cross validation with a better parametrized SVM. The results were much much better than before thankfully but still with my position in the middle of the leaderboard, I think there is a need for a lot more new ideas.
<p>In my next attempt, I will do a data deep dive. Idea would be to analyze where predictions are going wrong and also to think about what might make better features.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.display import display

In [3]:
Training_data = pd.read_json("../input/whatscooking/train.json")
Training_data['str_ingredients'] = Training_data.ingredients.apply(lambda x: ','.join(y for y in x))
y_Train = Training_data.cuisine
#describe_dataset(Training_data, "Training");
#Testing_data = pd.read_json("../input/whatscooking/test.json")
#Testing_data['str_ingredients'] = Testing_data.ingredients.apply(lambda x: ','.join(y for y in x))

In [4]:
import itertools
grouped_ingredients = itertools.groupby(sorted(list(itertools.chain.from_iterable(Training_data.ingredients))))
ingredient_counts = {key:len(list(group)) for key, group in grouped_ingredients}
global_pantry = pd.DataFrame.from_dict(ingredient_counts, orient='index')
global_pantry.columns = ['num_recipes']
display(global_pantry.count())
display(global_pantry.head())

num_recipes    6714
dtype: int64

,num_recipes
low-sodium fat-free chicken broth,22
sweetened coconut,3
baking chocolate,3
egg roll wrappers,57
bottled low sodium salsa,1


In [5]:
from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(sublinear_tf=True, vocabulary=global_pantry.index.values)

In [6]:
from sklearn.cross_validation import train_test_split
train, test = train_test_split(Training_data, test_size = 0.2)

X_train = vectorizer.fit_transform(train.str_ingredients)
y_train = train.cuisine

X_test = vectorizer.transform(test.str_ingredients)
y_test = test.cuisine

#X_Test = vectorizer.transform(Testing_data.str_ingredients)
#y_Test = Testing_data.cuisine

feature_names = vectorizer.get_feature_names()

In [8]:
from sklearn.svm import LinearSVC
classifier = LinearSVC(loss='squared_hinge', penalty='l2', dual=False, tol=1e-3)

In [9]:
from sklearn import metrics
from time import time

t0 = time()
classifier.fit(X_train, y_train)
train_time = time() - t0

t0 = time()
predictions = classifier.predict(X_test)
test_time = time() - t0

score = metrics.accuracy_score(y_test, predictions)

print "score: {}, train_time: {}, test_time: {}".format(score, train_time, test_time)

score: 0.734632306725, train_time: 3.99665904045, test_time: 0.00522708892822
